# 3.3 Hill's System

## Exercises 3.3

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

import numpy as np
from src.hills_system import matrix_inverse_mod26, validate_hills_key
from src.helpers import CHARACTERS, strip_text, format_ciphertext, format_plaintext, pos, char_at

### 5. Write a computer program to both encipher and decipher a message using Hill's (digraph) System.

We implement the general case which includes $2\times2$ (digraph).  See [hills_system.py](https://github.com/dandoug/cryptomath-book/blob/main/src/hills_system.py) for the validation logic and the computation of the inverse key.

In [2]:
class HillsCipher:
    def __init__(self, key: np.ndarray, pad_char: str = 'q'):
        validate_hills_key(key)
        self.key = key
        self.n = self.key.shape[0]
        self._init_inverse()
        self.pad_char = pad_char.strip().lower()
        if len(self.pad_char) != 1 or self.pad_char not in CHARACTERS:
            raise ValueError("pad_char must be a single character")

    def _init_inverse(self):
        try:
            self.inv_key = matrix_inverse_mod26(self.key)
        except ValueError as e:
            raise ValueError("Key is not invertible mod 26") from e


    def encipher(self, plaintext: str) -> str:
        """
        Encipher a message using Hill's cipher system.
        """
        plaintext = strip_text(plaintext)
        #  pad plaintext if the length is a multiple of n
        padding_length = (self.n - (len(plaintext) % self.n)) % self.n
        plaintext += self.pad_char * padding_length
        ciphertext = ""
        for i in range(0, len(plaintext), self.n):
            ngram = np.array([[pos(c)] for c in plaintext[i:i + self.n]])
            ciphertext_ngram = np.dot(self.key, ngram) % 26
            ciphertext += ''.join([char_at(int(c[0])) for c in ciphertext_ngram])
        return ciphertext.upper()

    def decipher(self, ciphertext: str) -> str:
        ciphertext = strip_text(ciphertext)
        if len(ciphertext) % self.n != 0:
            raise ValueError("Ciphertext length must be a multiple of n")
        plaintext = ""
        for i in range(0, len(ciphertext), self.n):
            ciphertext_ngram = np.array([[pos(c)] for c in ciphertext[i:i + self.n]])
            plaintext_ngram = np.dot(self.inv_key, ciphertext_ngram) % 26
            plaintext += ''.join([char_at(int(c[0])) for c in plaintext_ngram])
        return plaintext



### 1. Using Hill's System with $key = \begin{pmatrix} 6 & 3 \\ 7 & 8 \end{pmatrix}$ encipher the message:
```
 It is lonely at the top; but you eat better.
```

In [3]:
cipher_1 = HillsCipher(np.array([[6, 3], [7, 8]]))
plaintext_1 = "It is lonely at the top; but you eat better."

In [4]:
ciphertext_1 = cipher_1.encipher(plaintext_1)
print(format_ciphertext(ciphertext_1))

JOGGM VUHQX NKNVL MHYWZ MBWMG QVZLM EXCB


### 2. Decipher the following message that was enciphered using Hill's System with $key = \begin{pmatrix} 3 & 2 \\ 8 & 5 \end{pmatrix}$
```
 MUBYA QIQGN AEWOS RZQJI RZQKC LIZAG SXCJA AQFRM HO
```

In [5]:
cipher_2 = HillsCipher(np.array([[3, 2], [8, 5]]))
ciphertext_2 = "MUBYA QIQGN AEWOS RZQJI RZQKC LIZAG SXCJA AQFRM HO"

In [6]:
plaintext_2 = cipher_2.decipher(ciphertext_2)
print(format_plaintext(plaintext_2))

consciousnessisthatannoyingtimebetweennaps


which yields
```
  consciousness is that annoying time between naps
```

### 4. Consider extending Hill's (digraph) System to a trigraph system. That is, rather than enciphering and deciphering characters in pairs, you wish to encipher them three at a time. Describe how you would proceed. Be sure to consider whatever restrictions are necessary to impose on the key to assure that a message is decipherable. (Hint: See Exercises 3.2, #9.)

The determinent of the $3 \times 3$ matrix must be coprime to 26.  Plaintext must be padded to multiple of 3.  See general implementation above which will work for trigraph systems.

###  3. Molly has intercepted the following message that was enciphered using Hill's System (digraph version):

```
 KFHYY GIGMC EJSST EBOEU GRWJT SDVYK ZOZLI ZKFHX KUUIC WXFWJ
 GAXQP BQAGV GXDVD GUEVG MIGYK QQPIP SCLLF YPMUL KFHXP MHGME
 VDKAV YQCEG UEALY YYZSZ MPXZO CTXTR IMDID VDGSX OZFFT SMEDV
 MEIMD VMPKO UJKOD UBOAX BOORS LPZCW IMDVY GJWMI FQ
```

Help her cryptanalyze and decipher it.

Because for a $n \times n$ matrix, $A$, $A \times A^{-1} = A^{-1} \times A = I$ where $I$ is the identity matrix, we don't strictly need `CryptoAnalysisHillsCipher` class, but having one will let us define an instance with the decrypting key and still use the appropriate method names.  It's purely for readability.

In [9]:
class CryptoanalysisHillsCipher(HillsCipher):
    """
    Define a HillsCipher object with a conjecture of the decrypting key
    """
    def __init__(self, inv_key: np.ndarray, pad_char: str = 'q'):
        self.inv_key = inv_key  # Store inverse first
        try:
            key = matrix_inverse_mod26(inv_key)
        except ValueError as e:
            raise ValueError("Key is not invertible mod 26") from e
        super().__init__(key, pad_char=pad_char)

    def _init_inverse(self):
        # Override to do nothing since we already have the inverse
        pass